In [196]:
import os
import sys
import pandas as pd
from py2cytoscape import cyrest
from IPython.display import Image
%matplotlib inline
from py2cytoscape.cyrest.base import api

### check cytoscape connection and setup

In [197]:
cytoscape=cyrest.cyclient()
cytoscape.version()

apiVersion v1
cytoscapeVersion 3.7.2


In [198]:
cytoscape.session.new()

In [199]:
cytoscape.vizmap.apply(styles='default')

{'views': []}

## filter data

#### filter expression dataset

In [200]:
times = pd.read_csv('./data/Bendjilali2017G3.tsv',sep='\t')
print(times.shape)
times = times[
    (times['autocor.adj']<0.05) &
    (times['DESeq.adj']<0.05)
]
print(times.shape)
times.to_csv('./data/filtered_exp.tsv',sep='\t',index=False)

(7140, 11)
(467, 11)


#### select nodes with significant expression

In [201]:
network = pd.read_csv('./data/Yu2008Science.tsv',sep='\t',header=None)
print(network.shape)
network = network[(network[0].isin(times['gene'])) & (network[1].isin(times['gene']))]
print(network.shape)
network.to_csv('./data/network_filtered.tsv',index=False,header=False, sep='\t')

(1809, 3)
(33, 3)


## Load data into cytoscape

#### load network

In [202]:
api(namespace='network',command='import file',
    PARAMS ={
     "file": "/home/joeri/Documents/coding_projects/cytohack/data/network_filtered.tsv",
    "indexColumnSourceInteraction": "1",
    "indexColumnTargetInteraction": "2",
    "indexColumnTypeInteraction": "3",
    }
)

{'networks': [240], 'views': [319]}

#### load time series table into omicsvisualizer

In [203]:
api(namespace='ov',command='load',
    PARAMS ={
     "file": "/home/joeri/Documents/coding_projects/cytohack/data/filtered_exp.tsv",
    "keyColumnIndex": "1",
    "startLoadRow":'1',
    "dataTypeList":"s,d,d,d,d,d,d,d,d,d,d",
    "newTableName":'times',
    }
)


#### connect the ov table to the network

In [204]:
api(namespace='ov',command='connect',
    PARAMS ={
    'mappingColNet':'shared name',
    'mappingColTable':'gene',
    }
)

### create donut visualization of time series expression data

In [205]:
api(namespace='ov',command='viz apply outer continuous',
    PARAMS ={
    "attributes":'min0,min5,min10,min30,min60,min120,min180,min240',
    "transpose":'true',
    "rangeMax":'4000',
    "rangeMid":'2000',
    "rangeMin":'0',
    }
)


### NOT USED: filter the omics visualizer table
did the filtering in python, but kept this as a reference

**filter for autocor.adj**

In [129]:
api(namespace='ov',command='filter',
   PARAMS={
       'filter':"(autocor.adj,LOWER,0.05)",
       'tableName':'times'
   })

**filter for DESeq.adj**

In [130]:
api(namespace='ov',command='filter',
   PARAMS={
       'filter':"(DESeq.adj,LOWER,0.05)",
       'tableName':'times'
   })